### [How to chain runnables](https://python.langchain.com/docs/how_to/sequence/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Model

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")

chain = prompt | model | StrOutputParser()

# chain.invoke({"topic": "programacion"})
chunks = []
for chunk in chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|¡|Claro|!| Aquí| tienes| uno|:

|¿Por| qué| los| program|adores| pref|ieren| el| café| oscuro|?

|¡|Porque| les| gusta| tener| menos| bugs|!| ☕|🐛|

|Espero| que| te| haya| sac|ado| una| sonrisa|.| Si| quieres| otro|,| ¡|d|ím|elo|!||

#### B. Coercion

In [5]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

# composed_chain.invoke({"topic": "programacion"})
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|Este| ch|iste| juega| con| un| par| de| conceptos| que| son| comunes| en| el| ámbito| de| la| programación| y| la| cultura| de| los| program|adores|.| Vamos| a| des|gl|os|arlo|:

|1|.| **|C|afé| oscuro|**|:| La| referencia| al| café| oscuro| su|gi|ere| que| los| program|adores|,| que| a| menudo| se| as|oc|ian| con| largas| horas| de| trabajo|,| pref|ieren| una| bebida| que| les| mant|enga| alert|as| y| despi|ertos|.| El| café| es| una| bebida| popular| en| la| comunidad| de| program|adores|.

|2|.| **|L|uz| y| bugs|**|:| La| parte| más| ingen|iosa| del| ch|iste| es| la| al|usión| a| los| "|bugs|"| (|erro|res| o| fall|os| en| el| software|)| y| cómo| la| luz| atra|e| a| los| insect|os|.| En| la| programación|,| "|bugs|"| se| refiere| a| problemas| en| el| código| que| deben| solucion|arse|.| La| idea| de| que| "|la| luz| atra|e| a| los| bugs|"| crea| un| juego| de| palabras| que| conecta| la| idea| literal| de| los| insect|os| siendo| atra|ídos| por| la| luz| y| los| errores| en| el| c

#### C. Test

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Paso 1: Crear el primer chain para generar un chiste
joke_prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")
joke_chain = joke_prompt | model | StrOutputParser()

# Paso 2: Crear el segundo chain para analizar el chiste
analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}")
composed_chain = {"joke": joke_chain} | analysis_prompt | model | StrOutputParser()

# Paso 3: Realizar el streaming del composed_chain
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)


|Este| ch|iste| juega| con| dos| conceptos| que| son| comunes| en| el| ámbito| de| la| programación| y| la| cultura| de| los| program|adores|.| 

|1|.| **|C|afé| oscuro|**|:| Esta| parte| del| ch|iste| hace| referencia| a| la| prefer|encia| por| el| café| más| fuerte| o| con| un| sabor| más| intenso|,| que| es| un| est|ere|otipo| asociado| a| muchos| program|adores| que| suelen| consumir| café| para| mantenerse| despi|ertos| y| concentr|ados| durante| largas| horas| de| trabajo|.

|2|.| **|"|Byte|"|**|:| Aquí| se| hace| un| juego| de| palabras|.| En| el| ámbito| de| la| informática|,| un| byte| es| una| unidad| de| datos|.| Sin| embargo|,| la| palabra| "|byte|"| su|ena| similar| a| "|bite|"| (|m|ord|isco| en| inglés|),| lo| que| añade| un| doble| sentido| al| ch|iste|.| La| b|roma| su|gi|ere| que| los| program|adores| pref|ieren| el| café| oscuro| porque| "|t|ienen| un| buen| byte|",| insinu|ando| que| disfr|utan| tanto| del| café| como| de| la| programación|.

|El| humor| prov|iene| d

In [15]:
events = []
async for event in composed_chain.astream_events({"topic": "programacion"}, version="v2"):
    events.append(event)

event_types = {event["event"] for event in events}
print("Unique event types:", event_types)

Unique event types: {'on_chat_model_start', 'on_parser_stream', 'on_chain_end', 'on_chat_model_stream', 'on_chain_start', 'on_parser_start', 'on_prompt_start', 'on_chain_stream', 'on_prompt_end', 'on_parser_end', 'on_chat_model_end'}


In [7]:
events = [
    event
    async for event in composed_chain.astream_events(
        {"topic": "programacion"},
        version="v2",
    )
]

In [8]:
events

[{'event': 'on_chain_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': 'b3b4240d-1c32-4a56-97bb-feea0a41d319',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableParallel<joke>',
  'tags': ['seq:step:1'],
  'run_id': '69890a82-fd06-4152-a434-cfb3e14be6a4',
  'metadata': {},
  'parent_ids': ['b3b4240d-1c32-4a56-97bb-feea0a41d319']},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableSequence',
  'tags': ['map:key:joke'],
  'run_id': '701f6408-72e3-43c3-9434-0db1381d2284',
  'metadata': {},
  'parent_ids': ['b3b4240d-1c32-4a56-97bb-feea0a41d319',
   '69890a82-fd06-4152-a434-cfb3e14be6a4']},
 {'event': 'on_prompt_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'ChatPromptTemplate',
  'tags': ['seq:step:1'],
  'run_id': '79d52ae9-c969-4f03-8311-ce9508d49949',
  'metadata': {},
  'parent_ids': ['b3b4240d-1c32-4a56-97bb-feea0a41d319',
   '69890a82-fd06-

In [9]:
num_events = 0

async for event in composed_chain.astream_events({"topic": "programacion"}, version="v2"):
    kind = event["event"]
    
    if kind == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        print(f"Chat model chunk: {repr(chunk)}", flush=True)

    elif kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(f"Parser chunk: {chunk}", flush=True)

    elif kind == "on_chain_stream":
        chunk = event["data"].get("chunk", None)
        if isinstance(chunk, dict):
            for key, value in chunk.items():
                print(f"Chain {key} chunk: {repr(value)}", flush=True)
        else:
            print(f"Chain chunk: {repr(chunk)}", flush=True)

    num_events += 1

    if num_events > 30:  # Truncate output if too many events
        print("...", flush=True)
        break

Chat model chunk: ''
Parser chunk: 
Chain chunk: ''
Chain joke chunk: ''
Chat model chunk: '¡'
Parser chunk: ¡
Chain chunk: '¡'
Chain joke chunk: '¡'
Chat model chunk: 'Claro'
Parser chunk: Claro
Chain chunk: 'Claro'
Chain joke chunk: 'Claro'
Chat model chunk: '!'
Parser chunk: !
Chain chunk: '!'
Chain joke chunk: '!'
Chat model chunk: ' Aquí'
Parser chunk:  Aquí
Chain chunk: ' Aquí'
Chain joke chunk: ' Aquí'
Chat model chunk: ' tienes'
Parser chunk:  tienes
Chain chunk: ' tienes'
Chain joke chunk: ' tienes'
...


#### D. Metodo pipe

In [10]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

'Este chiste juega con el concepto de los Cylons en "Battlestar Galactica", una serie de ciencia ficción donde los Cylons son máquinas inteligentes que pueden imitar a los humanos. La clave del chiste radica en la palabra "trucos", que puede referirse tanto a las estrategias que se utilizan en el juego de cartas como a la idea de que los Cylons son muy buenos en el engaño y la manipulación.\n\nLa broma se basa en la ambigüedad de la palabra "trucos". En un sentido, se refiere a las jugadas que un jugador puede usar en un juego de cartas, pero también insinúa que los Cylons, al ser seres artificiales, no pueden ocultar sus verdaderas intenciones o habilidades, lo que hace que se "descubran" fácilmente en el contexto del juego.\n\nEl humor también proviene del contraste entre la idea de un juego de cartas, que suele ser un pasatiempo social y divertido, y la seriedad y la tensión que rodean la relación entre humanos y Cylons en la serie. Este tipo de chiste puede ser especialmente apreci

In [11]:
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)

chunks = []
for chunk in composed_chain_with_pipe.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|Este| ch|iste| juega| con| dos| conceptos| que| son| familiares| tanto| para| program|adores| como| para| quienes| disfr|utan| del| café|.| 

|1|.| **|Referencia| al| café| oscuro|**|:| El| "|c|afé| oscuro|"| se| refiere| a| un| tipo| de| café| que| se| tu|esta| más| tiempo|,| lo| que| resulta| en| un| sabor| más| fuerte| y| intenso|.| Es| un| juego| de| palabras| que| ev|oca| la| prefer|encia| de| muchos| program|adores| por| ese| tipo| de| café|.

|2|.| **|Juego| de| palabras| con| "|c|ódigo| sin| errores|"|**|:| Aquí| está| la| clave| del| ch|iste|.| En| programación|,| "|c|ódigo| sin| errores|"| significa| que| el| código| no| tiene| bugs| o| fall|os|,| lo| cual| es| un| objetivo| primordial| para| los| program|adores|.| Sin| embargo|,| el| ch|iste| su|gi|ere| que| al| igual| que| pref|ieren| su| café| oscuro|,| también| desean| que| su| código| esté| libre| de| errores|.

|La| combinación| de| estos| dos| elementos| crea| una| conexión| humor|ística| que| es| especialmente| aprec